## Work
1. 試改變 monitor "Validation Accuracy" 並比較結果
2. 調整 earlystop 的等待次數至 10, 25 並比較結果

In [ ]:
import os
import keras
import itertools
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

# Disable GPU
#os.environ["CUDA_VISIBLE_DEVICES"] = ""
#force to use cpu(by changing device_count in the tf.ContigProto)
import tensorflow as tf
from keras import backend as K
num_cores = 4
config = tf.ConfigProto(intra_op_parallelism_threads=num_cores,
                        inter_op_parallelism_threads=num_cores, 
                        allow_soft_placement=True,
                        device_count = {'CPU' : 1,
                                        'GPU' : 0}
                       )

session = tf.Session(config=config)
K.set_session(session)

In [ ]:
train, test = keras.datasets.cifar10.load_data()

In [ ]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [ ]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [ ]:
# write function to build model
def build_mlp(input_shape,output_units=10,num_neuron=[512,128,64]):
    model=Sequential()
    model.add(Dense(units=input_shape[1], input_dim=input_shape[1],kernel_initializer='normal',activation='relu',name='input'))
    model.add(BatchNormalization()) 
    for i, n_units in enumerate(num_neurons):
        model.add(Dense(units=n_units,kernel_initializer='normal',activation='relu',name='hidden'+str(i+1)))
        model.add(BatchNormalization()) 
    model.add(Dense(units=output_units,kernel_initializer='normal',activation='softmax',name='output'))
    return model

In [ ]:
# write function to plot results
def plot_results(results):
    color_bar=['maroon','red','sienna','greenyellow','lime','green','aqua','dodgerblue','mediumblue','slateblue','darkorchid','purple']
    """
    Draw the results
    """
    plt.figure(figsize=(12,9))
    for i, cond in enumerate(results.keys()):
        plt.plot(range(len(results[cond]['train_loss'])), results[cond]['train_loss'], '-', label='train'+cond, color=color_bar[i%12])
        plt.plot(range(len(results[cond]['valid_loss'])), results[cond]['valid_loss'],'--', label='valid'+cond, color=color_bar[i%12])
    plt.title('Loss')
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show

    plt.figure(figsize=(12,9))
    for i, cond in enumerate(results.keys()):
        plt.plot(range(len(results[cond]['train_acc'])), results[cond]['train_acc'], '-', label='train'+cond, color=color_bar[i%12])
        plt.plot(range(len(results[cond]['valid_acc'])), results[cond]['valid_acc'],'--', label='valid'+cond, color=color_bar[i%12])
    plt.title('Accuracy')
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()

In [ ]:
## 超參數設定
"""
Set your hyper-parameters
"""
LEARNING_RATE=1e-3
EPOCHS=30
BATCH_SIZE=256
MOMENTUM=0.9

## 作業一、改成monitor val_acc

In [ ]:
# 載入 Callbacks
"""
Earlystop setting ()
"""
from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor="val_acc", patience=5, verbose=1 )
"""
Your Training loop and Plot the results
"""
results_hw1={}
for monitor_opt in ['yes','no']:
    model=build_mlp(input_shape=x_train.shape)
    model.summary()
    optimizer=SGD(lr=LEARNING_RATE,nesterov=True, momentum=MOMENTUM)
    model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
    
    if monitor_opt =='yes':
        model.fit(x_train,y_train,
                  epochs=EPOCHS,
                  batch_size=BATCH_SIZE,
                  validation_data=(x_test,y_test),
                  shuffle=True,
                  callbacks=[earlystop])
    elif monitor_opt =='no':
        model.fit(x_train,y_train,
                  epochs=EPOCHS,
                  batch_size=BATCH_SIZE,
                  validation_data=(x_test,y_test),
                  shuffle=True)
    
    # collect results
    train_loss=model.history.history['loss']
    valid_loss=model.history.history['val_loss']
    train_acc=model.history.history['acc']
    valid_acc=model.history.history['val_acc']
    
    # construct dictionary
    exp_name_tag = 'exp_%s'%str('earlystop')+str(monitor)
        
    results_hw1[exp_name_tag]={'train_loss':train_loss,
                               'valid_loss':valid_loss,
                               'train_acc':train_acc,
                               'valid_acc':valid_acc}

In [ ]:
plot_results(results_hw1)

In [ ]:
## 作業二、調整 earlystop 的等待次數至 10, 25
WAITTIME=[5,10,25]
results_hw2={}
for waittime in WAITTIME:
    earlystop = EarlyStopping(monitor="val_acc", patience=waittime, verbose=1 )
    model=build_mlp(input_shape=x_train.shape)
    model.summary()
    optimizer=SGD(lr=LEARNING_RATE,nesterov=True, momentum=MOMENTUM)
    model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
    model.fit(x_train,y_train,
                  epochs=EPOCHS,
                  batch_size=BATCH_SIZE,
                  validation_data=(x_test,y_test),
                  shuffle=True,
                  callbacks=[earlystop])
    
    # collect results
    train_loss=model.history.history['loss']
    valid_loss=model.history.history['val_loss']
    train_acc=model.history.history['acc']
    valid_acc=model.history.history['val_acc']
    
    # construct dictionary
    exp_name_tag = 'exp_%s'%str('wait-time')+str(waittime)
        
    results_hw2[exp_name_tag]={'train_loss':train_loss,
                               'valid_loss':valid_loss,
                               'train_acc':train_acc,
                               'valid_acc':valid_acc}

In [ ]:
plot_results(results_hw2)